<span style="color: orange; font-family: Calibri Light;">
  <h1><b>MODEL EVALUATION: BERTopic (custom_word2vec_model/HDBSCAN/slightly_prep_data)</b></h1>
</span>
<span style = "font-family: Calibri Light">

model evaluation of the fourth bert model.

what do I need?

- the test data labelled by the bert model
- the human labels, arranged in the same order as in the test label.

for two above, since we don't have date of creation, we'll use IDs to arrange the rows

In [1]:
import pandas as pd
import os
from glob import glob
import ast
import re
import spacy
import time

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score, classification_report, adjusted_rand_score
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Access dataset files
folder_path = os.path.join( "..", "../../bert_model_data") 
file_type = "*.csv"

# List of dataset file paths
document_path = glob(os.path.join(folder_path, file_type))

document_path

['../../../bert_model_data/bert_default_raw_text_topics_term.csv',
 '../../../bert_model_data/test_data_bert_raw_label.csv',
 '../../../bert_model_data/bert_train_lemma.csv',
 '../../../bert_model_data/human_and_bert_raw_labels.csv',
 '../../../bert_model_data/bert_default_raw_text_topics_term_labelled.csv',
 '../../../bert_model_data/test_data_human_label_bert_raw.csv',
 '../../../bert_model_data/posts_upvote_ratio.csv',
 '../../../bert_model_data/bert_sw_test_data.csv',
 '../../../bert_model_data/bert_train_no_prep.csv',
 '../../../bert_model_data/bert_model_1_label_train_data.csv',
 '../../../bert_model_data/bert-data-sw.csv',
 '../../../bert_model_data/test_data_lemma.csv',
 '../../../bert_model_data/bert_kmeans_test_data.csv',
 '../../../bert_model_data/bert_w2v_test_data.csv']

In [3]:
#import cleaned data

def list_converter(text):
    #to revert list->str conversion from pd.read_csv
    return ast.literal_eval(text)


model_labelled_data = pd.read_csv("../../../bert_model_data/bert_w2v_test_data.csv", converters ={'tokens':list_converter})
human_labelled_data = pd.read_csv("../../../bert_model_data/test_data_human_label_bert_raw.csv", converters ={'tokens':list_converter})
#test_data = test_data.drop(columns = ['index', 'clean_text', 'tokens', 'word_count'])

In [4]:
model_labelled_data.drop(columns = ['Unnamed: 0'], inplace = True)
model_labelled_data.head()

,text_type,ID,date_created,long_text,topic_description
0,submission,wbwik,2012-07-10 19:11:08,Only in Dubai! R8: BRAND NEW 2012 Audi R8 V8 -...,-1_food_car_money_got
1,comment,c5c2v3q,2012-07-10 22:36:50,"Surprise! Happy birthday! \n\nAh man, *anoth...",-1_food_car_money_got
2,comment,c5cc9sz,2012-07-11 08:48:34,I looked at the AD and it seems more like a sa...,2_zomato_talabat_ads_crypto
3,comment,c5cdavh,2012-07-11 10:15:52,It's a marketing post. But isn't it against du...,-1_food_car_money_got
4,submission,10embj,2012-09-24 23:02:05,Dubai cap my dad brought me! [FYSR],0_uae_sharjah_visa_salary


In [5]:
#split `topic_description` into two columns for topic number and topic description
model_labelled_data[['model_topic_no', 'model_topic_desc']] = model_labelled_data['topic_description'].str.split('_', n = 1, expand = True)
model_labelled_data['model_topic_no'] = model_labelled_data['model_topic_no'].astype(int) 
model_labelled_data.drop(columns = ['topic_description'], inplace = True)

In [6]:
#sort by `ID`
model_labelled_data.sort_values(by=['ID'], ignore_index = True, inplace = True)
model_labelled_data.head()

,text_type,ID,date_created,long_text,model_topic_no,model_topic_desc
0,submission,10embj,2012-09-24 23:02:05,Dubai cap my dad brought me! [FYSR],0,uae_sharjah_visa_salary
1,submission,13ch705,2023-05-09 10:04:19,Anyone here have any idea where I can park my ...,8,speed_lanes_limit_parking
2,submission,13cpma4,2023-05-09 17:17:49,"UAE real estate agents and brokers, I need you...",0,uae_sharjah_visa_salary
3,submission,13p1nlq,2023-05-23 00:07:39,A Well Deserved Shout-out I posted in this sub...,2,zomato_talabat_ads_crypto
4,submission,14d7052,2023-06-19 10:52:36,Furnished apartments… bedding/pots and pans Hi...,4,rent_estate_agent_month


In [7]:
model_labelled_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   text_type         1200 non-null   object
 1   ID                1200 non-null   object
 2   date_created      1200 non-null   object
 3   long_text         1200 non-null   object
 4   model_topic_no    1200 non-null   int64 
 5   model_topic_desc  1200 non-null   object
dtypes: int64(1), object(5)
memory usage: 56.4+ KB


In [8]:
#dataset showing the human labels
human_labelled_data.head()

,ID,long_text,topics,topic_number_bert_raw,topic_number_bert_w2v,topic_number_kmeans,topic_number_bert_sw
0,10embj,Dubai cap my dad brought me! [FYSR],undefined,-1,-1,-1,-1
1,13ch705,Anyone here have any idea where I can park my ...,infrastructure.transportation,32,36,20,37
2,13cpma4,"UAE real estate agents and brokers, I need you...",real_estate_housing,24,4,5,12
3,13p1nlq,A Well Deserved Shout-out I posted in this sub...,shopping,61,61,61,61
4,14d7052,Furnished apartments‚Ä¶ bedding/pots and pans ...,real_estate_housing,24,4,5,12


In [9]:
#select only columns needed for this model
human_labelled_data = human_labelled_data[
    ['ID', 'long_text', 'topics', 'topic_number_bert_w2v']].copy()

In [10]:
human_labelled_data.head()

,ID,long_text,topics,topic_number_bert_w2v
0,10embj,Dubai cap my dad brought me! [FYSR],undefined,-1
1,13ch705,Anyone here have any idea where I can park my ...,infrastructure.transportation,36
2,13cpma4,"UAE real estate agents and brokers, I need you...",real_estate_housing,4
3,13p1nlq,A Well Deserved Shout-out I posted in this sub...,shopping,61
4,14d7052,Furnished apartments‚Ä¶ bedding/pots and pans ...,real_estate_housing,4


In [11]:
#include column in model_labelled_data to hold the human topic numbers

model_labelled_data[['human_topic_desc','human_topic_no']] = human_labelled_data[['topics','topic_number_bert_w2v']].copy()

In [12]:
model_labelled_data.head()

,text_type,ID,date_created,long_text,model_topic_no,model_topic_desc,human_topic_desc,human_topic_no
0,submission,10embj,2012-09-24 23:02:05,Dubai cap my dad brought me! [FYSR],0,uae_sharjah_visa_salary,undefined,-1
1,submission,13ch705,2023-05-09 10:04:19,Anyone here have any idea where I can park my ...,8,speed_lanes_limit_parking,infrastructure.transportation,36
2,submission,13cpma4,2023-05-09 17:17:49,"UAE real estate agents and brokers, I need you...",0,uae_sharjah_visa_salary,real_estate_housing,4
3,submission,13p1nlq,2023-05-23 00:07:39,A Well Deserved Shout-out I posted in this sub...,2,zomato_talabat_ads_crypto,shopping,61
4,submission,14d7052,2023-06-19 10:52:36,Furnished apartments… bedding/pots and pans Hi...,4,rent_estate_agent_month,real_estate_housing,4


In [13]:
model_labelled_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   text_type         1200 non-null   object
 1   ID                1200 non-null   object
 2   date_created      1200 non-null   object
 3   long_text         1200 non-null   object
 4   model_topic_no    1200 non-null   int64 
 5   model_topic_desc  1200 non-null   object
 6   human_topic_desc  1200 non-null   object
 7   human_topic_no    1200 non-null   int64 
dtypes: int64(2), object(6)
memory usage: 75.1+ KB


In [14]:
model_labelled_data.to_csv('human_and_model_labels_model4.csv')

In [15]:
len(model_labelled_data['human_topic_desc'].unique())

71

In [16]:
mask = model_labelled_data.human_topic_no ==24
model_labelled_data[mask]

,text_type,ID,date_created,long_text,model_topic_no,model_topic_desc,human_topic_desc,human_topic_no


<h2> ACCURACY SCORE </h2>

<h3>Accuracy score of the entire model prediction</h3>

In [17]:
#accuracy score
y_true = model_labelled_data.human_topic_no
y_pred = model_labelled_data.model_topic_no
accuracy = accuracy_score(y_true, y_pred, normalize = True)

print (f'Bertopic model with w2v and on slightly pre-preprocessed data\nModel accuracy is: {accuracy:.3f}')

Bertopic model with w2v and on slightly pre-preprocessed data
Model accuracy is: 0.359


<h3>Accuracy score of the individual human labels</h3>

In [18]:
topic_range = sorted(model_labelled_data['human_topic_no'].unique())
accuracy = {'topic_no':[],
            'human_topic_desc':[],
           'accuracy_score':[]
           }

for topic_no in topic_range:
    accuracy['topic_no'].append(topic_no) 
    mask = model_labelled_data.human_topic_no == topic_no
    topic_df = model_labelled_data[mask]
    
    accuracy['human_topic_desc'].append(topic_df['human_topic_desc'].unique())
    
    #accuracy
    y_true = topic_df.human_topic_no
    y_pred = topic_df.model_topic_no
    acc = accuracy_score(y_true, y_pred, normalize = True)
    
    accuracy['accuracy_score'].append(acc)

In [19]:
accuracy_df = pd.DataFrame(accuracy)
accuracy_df.set_index('topic_no', inplace = True)
accuracy_df.sort_values(by = 'accuracy_score', ascending = False)

,human_topic_desc,accuracy_score
topic_no,,
33,[fitness],1.0
13,[arabic_language],1.0
42,[sexual_solicitation],1.0
23,[personal_hygiene],1.0
15,[animal_welfare],1.0
...,...,...
55,[culture],0.0
56,[diversity_inclusion.gender],0.0
57,[economy],0.0


In [20]:
pd.set_option('display.max_rows', None)
accuracy_df

,human_topic_desc,accuracy_score
topic_no,,
-2,"[unrelated_theme, shopping]",0.000000
-1,[undefined],0.586207
3,"[diversity_inclusion.race/nationality, diversi...",0.219512
4,"[real_estate_housing, real_estate_housing]",0.500000
5,[media_communication],0.466667
6,[weather_conditions],0.700000
7,"[dining_service.food, dining_service.teas]",0.177778
8,[urban_mobility.traffic],0.425532
9,[safety.law_and_order],0.230769


In [21]:
accuracy_df[accuracy_df.accuracy_score >=0.5]

,human_topic_desc,accuracy_score
topic_no,,
-1,[undefined],0.586207
4,"[real_estate_housing, real_estate_housing]",0.500000
6,[weather_conditions],0.700000
10,[infrastructure.mobile_communication_internet],0.500000
13,[arabic_language],1.000000
14,[vehicles],0.727273
15,[animal_welfare],1.000000
19,[islam_religion],0.692308
23,[personal_hygiene],1.000000


In [22]:
#confusion matrix
y_true = topic_df['human_label']#model_labelled_data['human_label']
y_pred = topic_df ['topic_no'] #model_labelled_data['topic_no']
labels = sorted(list(set(y_true)))
cm = confusion_matrix(y_true = y_true, y_pred = y_pred)
print (cm)

KeyError: 'human_label'

<h2> PRECISION, RECALL, F1-SCORE </h2>

In [ ]:
#accuracy score
y_true = model_labelled_data.human_topic_no
y_pred = model_labelled_data.model_topic_no
#labels = list(model_labelled_data['human_topic_no'].unique())
clr = classification_report(y_true, y_pred,zero_division = 0.0, output_dict = True)

In [ ]:
clr_df = pd.DataFrame(clr).T
clr_df